# Evaluation scores
Using the file ../eval/scores_resnet_spec_eval.csv I evaluate the model performance of the evaluation process, performed on the ASVSpoof2021 dataset.


## Computing the % of correct predictions

In [1]:
import csv
import pandas as pd
from src.utils import *

In [2]:
config_path = '../config/residualnet_train_config.yaml'
config = read_yaml(config_path)
df_eval = pd.read_csv(os.path.join('..', config['df_eval_path']))

In [3]:
csv_score_file = '../eval/scores_resnet_spec_eval.csv'  
scores_eval = pd.read_csv(csv_score_file)

In [4]:
print(f'The length of the evaluation file list is {len(df_eval)}\nThe length of the evaluation score list is {len(scores_eval)}')

The length of the evaluation file list is 611829
The length of the evaluation score list is 249431


example: <br>
higher log probability ==> predicted class
if [score 0]=-2 and [score 1]=-3 then it means that the predicted class is class 0 <br>
[score] = -2+3 = 1 > 0 --> class 0
<br>
###########
<br>
if [score 0]=-3 and [score 1]=-2 then it means that the predicted class is class 1 <br>
[score] = -3+2 = -1 < 0 --> class 1

In [5]:
# the score values .csv uses a space as a delimiter between the file path and the score

def convert_column_to_binary(csv_file):
    binary_list = []
    
    with open(csv_file, 'r') as file:
        for line in file:
            # Split each line based on space 
            parts = line.strip().split()
            value = float(parts[1])
            
            try:
                if value > 0:
                    binary_list.append(0)
                else:
                    binary_list.append(1)
            except ValueError:
                pass
        
    return binary_list

In [6]:
scores_binary_list = convert_column_to_binary(csv_score_file)

In [7]:
# column at which I have the GT labels
csv_column_index = 2
csv_GT_file = '../data/df_eval.csv'

with open(csv_GT_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)
    csv_GT_values = [int(row[csv_column_index]) for row in reader]
    
total_values = len(scores_binary_list)
identical_count = sum(1 for x,y in zip(scores_binary_list, csv_GT_values) if x==y)
percentage_identical = (identical_count/total_values)*100


In [8]:
print(f'The number of GT values is {len(csv_GT_values)}')
print(f'The nuber of scores we got is {len(scores_binary_list)}')

The number of GT values is 611829
The nuber of scores we got is 249432


In [9]:
percentage_identical

89.26841784534463

## Computingh the ROC and EER

In [10]:
from sklearn.metrics import roc_curve
import numpy as np

In [11]:
def compute_eer(y_true, y_score):
    # compute the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    
    # find threshold
    eer_threshold_idx = np.argmin(np.abs(fpr-(1-tpr)))
    eer_threshold = thresholds[eer_threshold_idx]
    
    # Calculate EER (FPR or TPR at the EER threshold)
    eer_fpr = fpr[eer_threshold_idx]
    eer_tpr = tpr[eer_threshold_idx]
    eer = (eer_fpr + (1 - eer_tpr)) / 2.0
    
    return eer, eer_threshold

In [12]:
eer, eer_threshold = compute_eer(csv_GT_values, scores_binary_list)

ValueError: Found input variables with inconsistent numbers of samples: [611829, 249432]

In [ ]:
eer

## Computing the confusion matrix
* TN true negative, actual class was 0 (BF) and predicted as 0
* FP false positive, actual class was 0, but predicted as 1 (deep fake)
* FN false negative, actual class was 1, but predicted as 0
* TP true positive, actual class was 1 and predicted as 1

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(csv_GT_values, scores_binary_list)
print(cm)

# [ TN    FP
#   FN    TP ]

## Computing the indices of wrong predictions

In [ ]:
def find_different_indices(list1, list2):
    if len(list1) != len(list2):
        raise ValueError
    diff_idx = [i for i, (x,y) in enumerate(zip(list1, list2)) if x!=y]
    return diff_idx

wrong_preds_idx = find_different_indices(csv_GT_values, scores_binary_list)

In [ ]:
wrong_preds_idx[:10]